## ChatBot
 - 트랜스포머를 활용한 챗봇
 - [(딥러닝을 이용한 자연어처리 입문)](https://wikidocs.net/89786)

### 1. 데이터 로드 및 탐색
 - 데이터 출처 : [송영숙님의 챗봇데이터](https://github.com/songys/Chatbot_data)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import re

In [2]:
file_path = './data'
train_data = pd.read_csv(file_path + '/ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [4]:
# 결측치 확인
train_data.isna().sum()

Q        0
A        0
label    0
dtype: int64

In [5]:
# ? , . !과 같은 특수문자 전처리
# ex) 12시 땡! => 12시 땡 !
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r'([?.!,])', r' \1', sentence)    # \1, \2: 첫 번째, 두 번째 그룹
    sentence = sentence.strip()
    questions.append(sentence)
    
answers = []
for sentence in train_data['A']:
    sentence = re.sub(r'([?.!,])', r' \1', sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [6]:
print(questions[:7])
print(answers[:7])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네', 'SD카드 망가졌어', 'SD카드 안돼']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .', '다시 새로 사는 게 마음 편해요 .', '다시 새로 사는 게 마음 편해요 .']


### 2. 사전 생성
 - 질문과 답변 데이터로부터 사전 생성
 - 서브워드 토크나이저 중 SubwordTextEncoder 사용

In [7]:
# 토크나이저 정의 및 사전 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)    # 최대 사전 크기는 8192(=2**13)

In [8]:
# 사전 크기 확인
tokenizer.vocab_size

8176

In [9]:
# 만들어진 서브워드 확인
tokenizer.subwords[10:20]

['이_', '을_', '잘_', '도_', ' . ', '고_', '요', '것_', '많이_', '안_']

In [10]:
# SOS, EOS 토큰 만들어주기
# SOS, EOS 토큰에 정수 할당
START_TKN, END_TKN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# SOS, EOS 토큰만큼 사전 크기 +2로 늘려주기
VOCAB_SIZE = tokenizer.vocab_size + 2

In [12]:
print('시작 토큰 번호 :',START_TKN)
print('종료 토큰 번호 :',END_TKN)
print('단어 집합의 크기 :',VOCAB_SIZE)
# 추후 패딩토큰 추가

시작 토큰 번호 : [8176]
종료 토큰 번호 : [8177]
단어 집합의 크기 : 8178


In [13]:
dir(tokenizer)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_from_token_counts',
 '_byte_encode',
 '_cache_size',
 '_filename',
 '_id_to_subword',
 '_init_from_list',
 '_max_subword_len',
 '_read_lines_from_file',
 '_subword_to_id',
 '_subwords',
 '_token_to_ids',
 '_token_to_ids_cache',
 '_token_to_subwords',
 '_tokenizer',
 '_write_lines_to_file',
 'build_from_corpus',
 'decode',
 'encode',
 'load_from_file',
 'save_to_file',
 'subwords',
 'vocab_size']

In [14]:
tokenizer.subwords

[' .',
 ' ?',
 '거예요',
 '수_',
 '게_',
 '너무_',
 '더_',
 '거_',
 '좋아하는_',
 '는_',
 '이_',
 '을_',
 '잘_',
 '도_',
 ' . ',
 '고_',
 '요',
 '것_',
 '많이_',
 '안_',
 '좋은_',
 '같아요',
 '한_',
 '좀_',
 '있어요',
 '싶어',
 '가_',
 '나_',
 '에_',
 '있을_',
 '지_',
 '해보세요',
 '은_',
 '사람_',
 '할_',
 '해',
 '같아',
 '네',
 '면_',
 '건_',
 ' !',
 '사람이_',
 '를_',
 '마세요',
 '다_',
 '하고_',
 '지',
 '하는_',
 '보세요',
 '죠',
 '어',
 '서_',
 '내가_',
 '의_',
 '다',
 '내_',
 '이제_',
 '마음이_',
 '나',
 '다른_',
 '썸_',
 '만_',
 '그_',
 '어떻게_',
 '있는_',
 '왜_',
 '싶다',
 '세요',
 '다시_',
 '시간이_',
 '수도_',
 '없어',
 '것도_',
 '또_',
 '좋을_',
 '오늘_',
 '정말_',
 '가',
 '이',
 '같이_',
 '네요',
 '될_',
 '해요',
 '자꾸_',
 '걸_',
 '있어',
 '하세요',
 '없어요',
 '일_',
 '제가_',
 '길_',
 '바랄게요',
 '로_',
 '까',
 '돼요',
 '하면_',
 '봐요',
 '할까',
 '때_',
 '저도_',
 '으로_',
 '먼저_',
 '있을까',
 '에서_',
 '주세요',
 '그런_',
 '헤어진지_',
 '이별_',
 '될까',
 '기_',
 '고',
 '진짜_',
 '나를_',
 '야',
 '마음을_',
 '여자친구가_',
 '기',
 '좋아요',
 '없는_',
 '계속_',
 '남자친구가_',
 '혼자_',
 '해도_',
 '이별',
 '못_',
 '드세요',
 '줄_',
 '않아요',
 '먹고_',
 '는데_',
 '좋죠',
 '하지_',
 '일이_',
 '힘든_

### 3. 데이터 인코딩
 - encode 메서드 이용해 문장 인코딩 진행
 - ex) 가족들이랑 어디 가지 ?  =>  [7878, 1156, 1055, 2]
 - decode 메서드 이용해 문장 디코딩 진행
 - ex) [7878, 1156, 1055, 2]  => 가족들이랑 어디 가지? 

In [13]:
# 인코딩 및 디코딩 테스트
sample = questions[32]
encoded_sample = tokenizer.encode(sample)
decoded_sample = tokenizer.decode(encoded_sample)
print('질문 샘플 :', sample)
print()
print('인코딩 과정:', sample, '->', encoded_sample)
print('디코딩 과정:', encoded_sample, '->', decoded_sample)


질문 샘플 : 가족들이랑 어디 가지 ?

인코딩 과정: 가족들이랑 어디 가지 ? -> [7878, 1156, 1055, 2]
디코딩 과정: [7878, 1156, 1055, 2] -> 가족들이랑 어디 가지 ?


In [14]:
# 인코딩 최대 길이 = 40
MAX_LENGTH = 40

# 전체 데이터를 인코딩하는 함수
def tokenize_and_filter(inputs, outputs):
    """
    토큰화 -> 인코딩 -> SOS 토큰과 EOS 토큰 추가 -> 패딩
    """
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
        sentence1 = START_TKN + tokenizer.encode(sentence1) + END_TKN
        sentence2 = START_TKN + tokenizer.encode(sentence2) + END_TKN

        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)

    # 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs

In [15]:
questions, answers = tokenize_and_filter(questions, answers)

In [16]:
print(type(questions))
questions.shape

<class 'numpy.ndarray'>


(11823, 40)

In [17]:
# 샘플 확인
print(questions[2])
print(answers[2])

[8176 7971 1434 4651 7952 3650   67 8177    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[8176 3398  776  131    1 8177    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [21]:
# 사전 저장
save_path = './CBot'
tokenizer.save_to_file(save_path + '/CBot_vocab')

### 4. 데이터셋 생성
 - tf.data.Dataset 이용

In [18]:
batch_size = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()    # 데이터셋을 메모리에 캐시
dataset = dataset.shuffle(BUFFER_SIZE)    # buffer_size만큼 가져와 셔플링
dataset = dataset.batch(batch_size)    # 데이터셋을 batch_size개의 batch로 나눈다
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
# prefetch : 학습중일때 다음 batch를 미리 준비한다. 데이터 로드시간을 줄일 수 있고, 마지막에 선언해준다.
# prefetch(1)은 1개 batch를 미리 준비. tf.data.experimental.AUTOTUNE는 batch를 텐서플로가 자동으로 결정

In [19]:
# 임의의 샘플에 대해서 [:, :-1]과 [:, 1:]이 어떤 의미를 가지는지 테스트해본다.
print(answers[0]) # 기존 샘플
print()
print(answers[:1][:, :-1]) # 마지막 패딩 토큰 제거하면서 길이가 39가 된다.
print()
print(answers[:1][:, 1:]) # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 길이는 역시 39가 된다.

[8176 3842   74 7893    1 8177    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]

[[8176 3842   74 7893    1 8177    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]

[[3842   74 7893    1 8177    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


### 5. 모델 정의
 - 트랜스포머 모델을 불러와 사용

In [20]:
# .py로 만들어둔 트랜스포머 모델 임포트
import transformer

# Hyperparameter 정의
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1
MAX_LENGTH = 40

model = transformer.transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [21]:
# Custom schedule 정의
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
    def get_config(self):
        config = {
            'd_model': self.d_model,
            'warmup_steps': self.warmup_steps,}
        
        return config
    
def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(y_true, y_pred)
    
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)    # elementwise multiply
    
    return tf.reduce_mean(loss)    # reduce_mean: 전체원소의 합을 개수로 나눈값 리턴

In [22]:
# 학습률과 
learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [23]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
185/185 [==============================] - 20s 64ms/step - loss: 1.4521 - accuracy: 0.0287
Epoch 2/50
185/185 [==============================] - 12s 64ms/step - loss: 1.1734 - accuracy: 0.0495
Epoch 3/50
185/185 [==============================] - 12s 63ms/step - loss: 1.0035 - accuracy: 0.0508
Epoch 4/50
185/185 [==============================] - 12s 66ms/step - loss: 0.9244 - accuracy: 0.0547
Epoch 5/50
185/185 [==============================] - 12s 65ms/step - loss: 0.8627 - accuracy: 0.0582
Epoch 6/50
185/185 [==============================] - 12s 66ms/step - loss: 0.8011 - accuracy: 0.0626
Epoch 7/50
185/185 [==============================] - 12s 64ms/step - loss: 0.7344 - accuracy: 0.0689
Epoch 8/50
185/185 [==============================] - 12s 65ms/step - loss: 0.6631 - accuracy: 0.0766
Epoch 9/50
185/185 [==============================] - 12s 64ms/step - loss: 0.5877 - accuracy: 0.0846
Epoch 10/50
185/185 [==============================] - 12s 64ms/step - loss: 0.510

In [24]:
# model.save(save_path + '/CBot_model')    # 에러발생. 
model.save_weights(save_path + '/CBot_weights')    # 정상적으로 저장 ok.

### 6. 챗봇 구현

In [32]:
def preprocess_sentence(sentence):
    """
    입력문장을 전처리하는 함수
    
    단어와 구두점 사이에 공백 추가.
    ex) 12시 땡! -> 12시 땡 !
    """
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [33]:
def evaluate(sentence):
    # 입력 문장 전처리
    sentence = preprocess_sentence(sentence)

    # 입력 문장에 시작 토큰과 종료 토큰을 추가
    sentence = tf.expand_dims(START_TKN + tokenizer.encode(sentence) + END_TKN, axis=0)
    output = tf.expand_dims(START_TKN, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

        # 현재 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TKN[0]):
            break

        # 현재 시점의 예측 단어를 output(출력)에 연결한다.
        # output은 for문의 다음 루프에서 디코더의 입력이 된다.
        output = tf.concat([output, predicted_id], axis=-1)

    # 단어 예측이 모두 끝났다면 output을 리턴.
    return tf.squeeze(output, axis=0)

In [45]:
# import time
# t = time.time()
print('단순 채팅봇 CBot입니다.')
print('End, Exit 입력시 채팅봇이 종료됩니다.')
print()

while True:
    # Input
    sentence = input('User: ')
    
    # End, Exit => end the program
    end_words = ['END', 'EXIT', '종료', '끝']
    if sentence.upper() in end_words:
        break
    
    # Answer
    prediction = evaluate(sentence)
    answer = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])
    answer = re.sub(r" ([?.!,])", r"\1", answer)    # 특수문자 전 후 공백 제거
    print(f"CBot: {answer}")

단순 채팅봇 CBot입니다.
End, Exit 입력시 채팅봇이 종료됩니다.
User: 햐
CBot:좋은 사람 만날 수 있을 거예요.
User: 종료
